<a href="https://colab.research.google.com/github/rickey-dong/Ergomax/blob/main/pose_quality_assessment_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/tensorflow/docs

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from numpy.linalg import norm

In [3]:
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

In [4]:
NOSE = 0
LEFT_EYE = 1
RIGHT_EYE = 2
LEFT_EAR = 3
RIGHT_EAR = 4
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6
Y = 0
X = 1
CONFIDENCE_VALUE = 2
CONFIDENCE_THRESHOLD = 0.26

In [21]:
def feature_extraction(image_file):
    """
    takes in an image
    and
    returns a list of relevant landmarks
    with y, x, and confidence values
    """

    # run the model!
    outputs = movenet(image_file)

    keypoints = outputs['output_0'].numpy()

    # converting this 4-layered array into something more manageable
    # i don't think we're too comfortable working with numpy so maybe
    # better to convert
    cleaner_2d_array = [ [0]*3 for i in range(7) ]
    body_part = 0
    for ary0 in keypoints:
        for ary1 in ary0:
            while body_part < 7:
                index = 0
                for num in keypoints[0][0][body_part]:
                    cleaner_2d_array[body_part][index] = num
                    index += 1
                body_part += 1

    return cleaner_2d_array

def calculate_cosine_similarity(ideal, current):
    ideal_1D = []
    for body_point in ideal:
      ideal_1D.append(body_point[1]) # x value
      ideal_1D.append(body_point[0]) # y value
    
    current_1D = []
    for body_point in current:
      current_1D.append(body_point[1])
      current_1D.append(body_point[0])
  
    A = np.array(ideal_1D)
    B = np.array(current_1D)
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine
  
def check_head_tilt(current):
    # if eyes are at ear level, then head is tilted
    if current[LEFT_EYE][Y] > current[LEFT_EAR][Y] + 0.10:
      return False
    if current[RIGHT_EYE][Y] > current[RIGHT_EAR][Y] + 0.10:
      return False
    return True

In [6]:
baseline_rickey = """
0.51898634 0.5141955 0.60066235 
0.46213895 0.5748569 0.8942077 
0.46386486 0.45547873 0.8542165 
0.5035565 0.6509616 0.5807533 
0.51048446 0.37870285 0.7500463 
0.78193563 0.7620751 0.5300895 
0.7900616 0.30074388 0.65386754
"""

baseline_rickey_array = [[0.51898634, 0.5141955, 0.60066235], [0.46213895, 0.5748569, 0.8942077], [0.46386486, 0.45547873, 0.8542165], [0.5035565, 0.6509616, 0.5807533], [0.51048446, 0.37870285, 0.7500463], [0.78193563, 0.7620751, 0.5300895], [0.7900616, 0.30074388, 0.65386754]]

slouch_rickey = """
0.59844774 0.519061 0.40708265 
0.5301816 0.59195983 0.8289351 
0.5341892 0.45447135 0.4498962 
0.57492197 0.67865723 0.6057053 
0.5780213 0.3705523 0.5283152 
0.7645539 0.8212724 0.2530607 
0.7899105 0.28375655 0.25369805
"""

current_rickey_array = [[0.59844774, 0.519061, 0.40708265], [0.5301816, 0.59195983, 0.8289351], [0.5341892, 0.45447135, 0.4498962], [0.57492197, 0.67865723, 0.6057053], [0.5780213, 0.3705523, 0.5283152], [0.7645539, 0.8212724, 0.2530607], [0.7899105, 0.28375655, 0.25369805]]

slouch_rickey_tilted_head = """
0.6696064 0.533864 0.71748036 
0.6088849 0.5900854 0.73862654 
0.6069633 0.47765368 0.6631635 
0.60519946 0.6635702 0.43899816 
0.60147464 0.4105199 0.5412193 
0.8310034 0.7259691 0.3184248 
0.83240736 0.34737605 0.37198436
"""

slouch_rickey_tilted_head_array = [[0.6696064, 0.533864, 0.71748036], [0.6088849, 0.5900854, 0.73862654], [0.6069633, 0.47765368, 0.6631635], [0.60519946, 0.6635702, 0.43899816], [0.60147464, 0.4105199, 0.5412193], [0.8310034, 0.7259691, 0.3184248], [0.83240736, 0.34737605, 0.37198436]]


In [ ]:
# baseline = tf.io.read_file("baseline-rickey.jpg")
# baseline = tf.compat.v1.image.decode_jpeg(baseline)
# baseline = tf.expand_dims(baseline, axis=0)
# baseline = tf.cast(tf.image.resize_with_pad(baseline, 192, 192), dtype=tf.int32)

# baseline_array = feature_extraction(baseline)
# for row in baseline_rickey_array:
#   string = ""
#   for number in row:
#     string += str(number)
#     string += " "
#   print(string)

# current = tf.io.read_file("slouch-rickey.jpg")
# current = tf.compat.v1.image.decode_jpeg(current)
# current = tf.expand_dims(current, axis=0)
# current = tf.cast(tf.image.resize_with_pad(current, 192, 192), dtype=tf.int32)

# current_array = feature_extraction(current)

# tilted = tf.io.read_file("slouch-rickey-tilted-head.jpg")
# tilted = tf.compat.v1.image.decode_jpeg(tilted)
# tilted = tf.expand_dims(tilted, axis=0)
# tilted = tf.cast(tf.image.resize_with_pad(tilted, 192, 192), dtype=tf.int32)

# slouch_rickey_tilted_head = feature_extraction(tilted)

# for row in slouch_rickey_tilted_head:
#   string = ""
#   for number in row:
#     string += str(number)
#     string += " "
#   print(string)

# print(slouch_rickey_tilted_head)

In [20]:
def has_bad_posture(ideal, current):
    """
    takes in two lists of keypoints and confidence values
    returns true if user has bad posture currently,
    false otherwise
    """
    # useful articles?
    # https://medium.com/roonyx/pose-estimation-and-matching-with-tensorflow-lite-posenet-model-ea2e9249abbd
    # https://medium.com/@priyaanka.garg/comparison-of-human-poses-with-posenet-e9ffc36b7427
    if current[NOSE][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD or \
        current[LEFT_EYE][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD or \
        current[RIGHT_EYE][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD or \
        current[LEFT_EAR][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD or \
        current[RIGHT_EAR][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD or \
        current[LEFT_SHOULDER][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD or \
        current[RIGHT_SHOULDER][CONFIDENCE_VALUE] < CONFIDENCE_THRESHOLD:
            # if the model is less than 26% confident about the location of any
            # particular body point, then there's not enough data to make a guess
            # or could mean that the user is severely slouching and has most of the
            # body off camera
            return 'NOT ENOUGH DATA'
    cos_sim = calculate_cosine_similarity(ideal, current)
    print(cos_sim)
    if cos_sim < 0.4:
      return 'NOT SIMILAR POSTURE'
    if check_head_tilt(current) == True:
      return 'HEAD TILTED'
    # if current body landmarks y-coords are greater than the baseline,
    # then that means the landmarks are further down than ideal, so could
    # be indicative of slouching
    # if ((current[LEFT_SHOULDER][Y] >= ideal[LEFT_SHOULDER][Y] + 0.01) or (current[RIGHT_SHOULDER][Y] >= ideal[RIGHT_SHOULDER][Y] + 0.01)):
    #     return 'BAD POSTURE 0'
    # if ((current[LEFT_EAR][Y] >= ideal[LEFT_EAR][Y] + 0.01) or (current[RIGHT_EAR][Y] >= ideal[RIGHT_EAR][Y] + 0.01)):
    #     return 'BAD POSTURE 1'
    return 'NICE'


In [22]:
print(has_bad_posture(baseline_rickey_array, slouch_rickey_tilted_head_array))

0.994963661516067
HEAD TILTED
